# Network Optimization Demo

## Import libraries

In [1]:
from network_optimizer import NetworkOptimizer
import synthetic_data_generators as sdg
import numpy as np
import pandas as pd
import time

## Import Synthetic Data

In [2]:
synth_network = pd.read_csv("../data/synth_network.csv")
synth_pool = pd.read_csv("../data/synth_pool.csv")
synth_members = pd.read_csv("../data/synth_members.csv")
synth_reqs = sdg.synth_reqs

### Pool of providers

In [3]:
print(f"Number of Access Points in the provider pool: {len(synth_pool)}")
print(f"Number of NPIs: {synth_pool.npi.nunique()}")
print(f"Number of Provider Groups (Contracting Entities): {synth_pool.group_id.nunique()}")
synth_pool.head()

Number of Access Points in the provider pool: 100
Number of NPIs: 66
Number of Provider Groups (Contracting Entities): 15


,npi,specialty,group_id,efficiency,effectiveness,location_id,county,latitude,longitude
0,14,urologist,2,5,1,0,wayne,41.916604,-83.793571
1,47,ent,7,3,1,1,wayne,42.331189,-83.668563
2,78,pcp,14,3,1,2,wayne,42.033343,-83.656534
3,60,pcp,7,5,1,3,wayne,42.223686,-83.744325
4,5,urologist,2,2,5,4,wayne,42.189181,-83.655711


### Members

In [4]:
print(f"Number of members: {len(synth_members)}")
synth_members.head()

Number of members: 1000


,member_id,county,latitude,longitude
0,0,wayne,42.334931,-83.798905
1,1,wayne,42.089999,-83.752170
2,2,wayne,42.044657,-83.500196
3,3,wayne,42.281720,-83.652579
4,4,wayne,42.498681,-83.769492


### Adequacy Requirements

In [5]:
synth_reqs

,specialty,county,provider_count,distance_req,min_access_pct,min_providers
0,cardiologist,wayne,1,15,90,5
1,pcp,wayne,2,15,90,10
2,ent,wayne,1,15,90,5
3,urologist,wayne,1,15,90,5
4,obgyn,wayne,1,15,90,5


## Run Optimization

### Create Network from scratch

In [6]:
optimizer = NetworkOptimizer(pool=synth_pool, 
                             members=synth_members,
                             adequacy_reqs=synth_reqs,
                             #no network
                             )

In [7]:
optimizer.optimize(5)

Optimization round 1 ...
Optimization round 2 ...
Optimization round 3 ...
Optimization round 4 ...
Optimization round 5 ...
Average seconds per round of optimization: 1.4
Adequacy Index for best network: 0.81



In [8]:
optimizer.adequacy_detail

,specialty,county,provider_count,distance_req,min_access_pct,min_providers,total_members,members_with_access,servicing_providers,pct_with_access,adequacy_index
0,cardiologist,wayne,1,15,90,5,1000,1000,9,1.000,1.0000
1,pcp,wayne,2,15,90,10,1000,964,9,0.964,0.8676
2,ent,wayne,1,15,90,5,1000,989,6,0.989,0.9890
3,urologist,wayne,1,15,90,5,1000,910,6,0.910,0.9100
4,obgyn,wayne,1,15,90,5,1000,680,2,0.680,0.2720


In [9]:
optimizer.move_tracker

[('addition', 4),
 ('addition', 2),
 ('addition', 14),
 ('addition', 0),
 ('addition', 10)]

In [10]:
optimizer.performance_history

array([0.     , 0.25128, 0.4768 , 0.65592, 0.74724, 0.80772])

#### Check moves

In [11]:
optimizer.best_network.loc[optimizer.best_network.group_id == optimizer.move_tracker[0][1]]

,npi,specialty,group_id,efficiency,effectiveness,location_id,county,latitude,longitude,pct_serving
0,72,cardiologist,4,5,1,6,wayne,42.000698,-83.751425,0.334
1,56,urologist,4,1,1,7,wayne,42.199648,-83.761273,0.628
2,89,cardiologist,4,1,1,28,wayne,42.153219,-83.988858,0.323
3,27,ent,4,1,2,45,wayne,42.286955,-83.598814,0.587
4,81,cardiologist,4,4,1,46,wayne,42.296682,-83.854589,0.548
5,72,cardiologist,4,5,1,48,wayne,42.290157,-83.631864,0.629
6,1,ent,4,2,1,49,wayne,41.938914,-83.417463,0.116
7,1,ent,4,2,1,53,wayne,42.157524,-83.947248,0.382
8,27,ent,4,1,2,54,wayne,42.000392,-83.790992,0.316
9,55,pcp,4,2,4,57,wayne,42.266743,-83.822039,0.594


### Create Network from scratch using custom objective
Optimizing for effectiveness!

In [14]:
def user_objective(self, network):
    if len(network) == 0:
        return 0
    else:
        return self.adequacy(network) * network.effectiveness.mean()

In [15]:
optimizer = NetworkOptimizer(pool=synth_pool, 
                             members=synth_members,
                             adequacy_reqs=synth_reqs,
                             # no network
                             user_objective=user_objective
                             )

In [16]:
optimizer.optimize(5)

Optimization round 1 ...
Optimization round 2 ...
Optimization round 3 ...
Optimization round 4 ...
Optimization round 5 ...
Average seconds per round of optimization: 1.3
Adequacy Index for best network: 0.81



In [17]:
optimizer.adequacy_detail

,specialty,county,provider_count,distance_req,min_access_pct,min_providers,total_members,members_with_access,servicing_providers,pct_with_access,adequacy_index
0,cardiologist,wayne,1,15,90,5,1000,1000,6,1.000,1.0000
1,pcp,wayne,2,15,90,10,1000,966,9,0.966,0.8694
2,ent,wayne,1,15,90,5,1000,954,6,0.954,0.9540
3,urologist,wayne,1,15,90,5,1000,872,5,0.872,0.8720
4,obgyn,wayne,1,15,90,5,1000,902,2,0.902,0.3608


In [18]:
optimizer.move_tracker

[('addition', 2),
 ('addition', 1),
 ('addition', 14),
 ('addition', 0),
 ('addition', 5)]

In [19]:
optimizer.performance_history

array([0.        , 0.62345   , 1.25028471, 1.866864  , 2.31783568,
       2.54961143])

In [20]:
optimizer.best_network.loc[optimizer.best_network.group_id == optimizer.move_tracker[0][1]]

,npi,specialty,group_id,efficiency,effectiveness,location_id,county,latitude,longitude,pct_serving
0,14,urologist,2,5,1,0,wayne,41.916604,-83.793571,0.175
1,5,urologist,2,2,5,4,wayne,42.189181,-83.655711,0.639
2,32,cardiologist,2,3,3,12,wayne,41.982906,-83.838536,0.268
3,62,urologist,2,2,2,22,wayne,42.113551,-83.472148,0.345
4,57,pcp,2,5,3,30,wayne,42.027369,-83.644040,0.359
5,5,urologist,2,2,5,39,wayne,42.361643,-83.532102,0.436
6,76,ent,2,5,5,63,wayne,42.396234,-83.950313,0.321
7,37,cardiologist,2,1,5,74,wayne,42.222695,-83.420171,0.356
8,37,cardiologist,2,1,5,78,wayne,42.061799,-83.649882,0.433
9,57,pcp,2,5,3,83,wayne,42.052663,-83.892604,0.324


### Optimize Existing Network 

In [21]:
test_existing_network = synth_pool.loc[synth_pool.group_id == 2]

In [22]:
optimizer = NetworkOptimizer(pool=synth_pool, 
                             members=synth_members,
                             adequacy_reqs=synth_reqs,
                             network=test_existing_network
                             )

In [23]:
optimizer.adequacy(test_existing_network)

0.17395999999999998

In [24]:
optimizer.adequacy_detail

,specialty,county,provider_count,distance_req,min_access_pct,min_providers,total_members,members_with_access,servicing_providers,pct_with_access,adequacy_index
0,cardiologist,wayne,1,15,90,5,1000,630.0,2.0,0.630,0.2520
1,pcp,wayne,2,15,90,10,1000,478.0,1.0,0.478,0.0478
2,ent,wayne,1,15,90,5,1000,321.0,1.0,0.321,0.0642
3,urologist,wayne,1,15,90,5,1000,843.0,3.0,0.843,0.5058
4,obgyn,wayne,1,15,90,5,1000,0.0,0.0,0.000,0.0000


In [25]:
optimizer.optimize(5)

Optimization round 1 ...
Optimization round 2 ...
Optimization round 3 ...
Optimization round 4 ...
Optimization round 5 ...
Average seconds per round of optimization: 1.4
Adequacy Index for best network: 0.87



In [26]:
optimizer.adequacy_detail

,specialty,county,provider_count,distance_req,min_access_pct,min_providers,total_members,members_with_access,servicing_providers,pct_with_access,adequacy_index
0,cardiologist,wayne,1,15,90,5,1000,1000,9,1.000,1.0000
1,pcp,wayne,2,15,90,10,1000,964,9,0.964,0.8676
2,ent,wayne,1,15,90,5,1000,989,6,0.989,0.9890
3,urologist,wayne,1,15,90,5,1000,910,7,0.910,0.9100
4,obgyn,wayne,1,15,90,5,1000,741,4,0.741,0.5928


In [27]:
optimizer.move_tracker

[('addition', 4),
 ('addition', 14),
 ('addition', 0),
 ('addition', 10),
 ('addition', 13)]

In [28]:
optimizer.performance_history

array([0.17396, 0.4822 , 0.65592, 0.74724, 0.80772, 0.87188])

In [29]:
optimizer.best_network.head()

,npi,specialty,group_id,efficiency,effectiveness,location_id,county,latitude,longitude,pct_serving
0,14,urologist,2,5,1,0,wayne,41.916604,-83.793571,0.175
1,5,urologist,2,2,5,4,wayne,42.189181,-83.655711,0.639
2,32,cardiologist,2,3,3,12,wayne,41.982906,-83.838536,0.268
3,62,urologist,2,2,2,22,wayne,42.113551,-83.472148,0.345
4,57,pcp,2,5,3,30,wayne,42.027369,-83.644040,0.359
